In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session
from snowflake.snowpark.types import StringType

session = get_active_session()


TO CONVERT merge_revenue_ifrs_dd_pi.py CODE

In [ ]:
from snowflake.snowpark import Session, DataFrame

def process_data(session: Session, event_date: str):
    source_table = "tsel_poc.raw.revenue_pi_dd_poc_tokenized"
    target_table = "telkomsel_poc.curated.merge_revenue_ifrs_dd_poc_tokenized"  # Replace with your actual target table

    # Step 1: Delete existing data for the event_date AND source = 'PI'
    delete_sql = f""" DELETE FROM {target_table} WHERE event_date = '{event_date}' AND source = 'PI' """

    session.sql(delete_sql).collect()

    # Compose SQL query
    query = f"""
    SELECT expiry_date_usage AS trx_date,
           purchase_date_2 AS purchase_date,
           transaction_id,
           '' AS subs_id,
           msisdn,
           CAST(reserve1 AS INT) AS price_plan_id,
           brand,
           1 AS pre_post_flag,
           cust_type AS cust_type_desc,
           cust_subtype AS cust_subtype_desc,
           '' AS customer_sub_segment,
           '' AS lac,
           '' AS ci,
           '' AS lacci_id,
           node_type AS node,
           'UNKNOWN' AS area_sales,
           CASE WHEN region IS NULL OR region = '' THEN 'UNKNOWN' ELSE region END AS region_sales,
           CASE WHEN branch IS NULL OR branch = '' THEN 'UNKNOWN' ELSE branch END AS branch,
           'UNKNOWN' AS subbranch,
           CASE WHEN cluster IS NULL OR cluster = '' THEN 'UNKNOWN' ELSE cluster END AS cluster_sales,
           'UNKNOWN' AS provinsi,
           'UNKNOWN' AS kabupaten,
           'UNKNOWN' AS kecamatan,
           'UNKNOWN' AS kelurahan,
           NULL AS lacci_closing_flag,
           bid AS sigma_business_id,
           '' AS sigma_rules_id,
           '' AS sku,
           '' AS l1_payu,
           '' AS l2_service_type,
           '' AS l3_allowance_type,
           '' AS l4_product_category,
           '' AS l5_product,
           l1_ias,
           l2_ias,
           l3_ias,
           '' AS commercial_name,
           '' AS channel,
           '' AS pack_validity,
           CAST(SUM(pi_value_final) AS DECIMAL(38,15)) AS rev_per_usage,
           CAST(SUM(0) AS DECIMAL(38,15)) AS rev_seized,
           CAST(SUM(0) AS INT) AS dur,
           CAST(SUM(0) AS INT) AS trx,
           CAST(SUM(0) AS BIGINT) AS vol,
           NULL AS cust_id,
           '' AS profile_name,
           '' AS quota_name,
           '' AS service_filter,
           '' AS price_plan_name,
           SUBSTR(transaction_id, 1, 2) AS channel_id,
           '' AS site_id,
           '' AS site_name,
           '' AS region_hlr,
           '' AS city_hlr,
           current_date() AS load_date ,
           expiry_date_usage AS event_date,
           'PI' AS source
    FROM {source_table}
    WHERE expiry_date_usage = '{event_date}'
    GROUP BY expiry_date_usage, purchase_date_2, transaction_id, msisdn, reserve1, brand, cust_type, cust_subtype, node_type, region, branch, cluster, bid, l1_ias, l2_ias, l3_ias, transaction_id
    """
    df = session.sql(query)

    # Step 3: Append the query result to the target table
    df.write.mode("append").save_as_table(target_table)

    return f"Deleted and appended data for event_date: {event_date} and source = 'SEIZED'"

In [ ]:
session.sproc.register(
    func=process_data,
    name="process_data_proc_merge_revenue_ifrs_dd_pi",
    is_permanent=True,
    stage_location="@TSELPOC_STAGE",           # Replace with your actual stage
    replace=True,
    input_types=[StringType()],          # Instead of [str]
    return_type=StringType(),            # Instead of str or "STRING"
    packages=["snowflake-snowpark-python"]
)


In [ ]:
call TELKOMSEL_POC.CURATED.process_data_proc_merge_revenue_ifrs_dd_pi('2025-04-01')

In [ ]:
select count(*) from telkomsel_poc.curated.merge_revenue_ifrs_dd_poc_tokenized;

TO CONVERT merge_revenue_ifrs_dd_seized.py Code